In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle

from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
import multiprocessing as mp
import importlib
from sklearn.model_selection import train_test_split


from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,Conv1D,TimeDistributed,MaxPooling1D,Flatten,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.layers import Bidirectional
from keras.regularizers import l2

from keras import backend as K
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [3]:
#tail -n +2 train.csv | split -l 150000


from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [4]:
#
columns = ['acoustic_data','time_to_failure']



In [ ]:
train_data, val_data = train_test_split(splits, test_size=0.1, random_state=42)

In [ ]:
TIMESTEPS=150000
BATCH_SIZE=16
NUMBER_OF_BATCHES = int(np.ceil(len(train_data)/BATCH_SIZE))
NUMBER_OF_VALIDATION_STEPS = int(np.ceil(len(val_data) / BATCH_SIZE))

In [ ]:
train_data_batch = np.array_split(train_data, NUMBER_OF_BATCHES)
val_data_batch = np.array_split(val_data, NUMBER_OF_VALIDATION_STEPS)

In [ ]:
NOISE=0.75

def add_noise(dff, pct=NOISE):
    mu = dff['acoustic_data'].mean()
    sigma = dff['acoustic_data'].std()

    indices = np.random.choice(dff.index.values, int(len(dff)*pct))
    dff.loc[indices, 'acoustic_data'] = np.random.normal(mu, sigma, len(indices)) 
    return dff


def get_batch(list_of_files, valid=False):
#     batch = np.empty((len(list_of_files),TIMESTEPS,1),dtype=float)
#     target = np.empty((len(list_of_files),1),dtype=float)
    #print(list_of_files)
    batch = []
    target = []

    for idx, file in enumerate(list_of_files):
        #print(idx,file)
        path = f'train/{file}'
        df = pd.read_csv(path, float_precision='round_trip', header=None)
        df.columns = columns
        df[['acoustic_data']] = StandardScaler().fit_transform(df[['acoustic_data']].astype('float'))
        #print(df.head())
        #print(len(batch))
        batch.append(df['acoustic_data'].values)
        target.append(df['time_to_failure'].values[-1])
        #print(df_noise.head())
        #if not valid:
            #df_noise = add_noise(df)
            #batch.append(df_noise['acoustic_data'].values)#.reshape(-1,TIMESTEPS,1))
            #target.append(df_noise['time_to_failure'].values[-1])#.reshape(-1,1))
        #print(np.array(batch).reshape(-1,TIMESTEPS,1).shape)
        #batch = np.array(batch).reshape(-1,TIMESTEPS,1)
        #target = np.array(target).reshape(-1,1)
    return (batch, target)

In [ ]:
from keras.utils import Sequence

class MY_Generator(Sequence):

    def __init__(self, list_of_files, steps,name):
        self.list_of_files = list_of_files
        self.steps = steps
        self.name = name

    #This function computes the number of batches that this generator is supposed to produce. 
    #So, we divide the number of total samples by the batch_size and return that value.    
    def __len__(self):
        return self.steps

    #Here, given the batch numberidx you need to put together a list that consists of data 
    #batch and the ground-truth (GT). In this example, we read a batch images of size 
    #self.batch and return an array of form[image_batch, GT]
    def __getitem__(self, idx):
#         if self.name == 'val':
        #print('idx', idx)
        #print("DEBUG", self.list_of_files[idx])
        #if idx == len(self.list_of_files):
            #print(idx, self.list_of_files)
        if self.name == 'val':
            valid = True
        else:
            valid=False
        train,Y = get_batch(self.list_of_files[idx], valid)
            
        #print(np.array(train).reshape(-1,TIMESTEPS,1).shape)
        #print("idx",idx)
        #print("LOLILOL")
        #print(train.shape, Y.shape)
        train = np.array(train).reshape(-1,TIMESTEPS,1)
        #print(train.shape)
        Y = np.array(Y).reshape(-1,1)

        return (train,Y)

In [ ]:
#train_data, val_data, y_train, y_val = train_test_split(training, targets, test_size=0.1, random_state=42)

K.clear_session()

TIMESTEPS=150000

dropout=0.33

kernel_size=5
filters=3
strides=5
pool_size=2
regularizer=l2(0.05)

my_model = Sequential()
my_model.add(
        Conv1D(filters=filters, kernel_size=kernel_size, activation="relu",
               kernel_regularizer=regularizer,
               strides=strides, input_shape=(TIMESTEPS,1))
)
             
my_model.add(MaxPooling1D(pool_size=pool_size))
my_model.add(BatchNormalization())

my_model.add(
        Conv1D(filters=filters, kernel_size=kernel_size, activation='relu',
               kernel_regularizer=regularizer,
               strides=strides, input_shape=(TIMESTEPS,1))
)             
my_model.add(MaxPooling1D(pool_size=pool_size))
my_model.add(BatchNormalization())

my_model.add(
        Conv1D(filters=filters, kernel_size=kernel_size, activation='relu',
               kernel_regularizer=regularizer,
               strides=strides, input_shape=(TIMESTEPS,1))
)             
my_model.add(MaxPooling1D(pool_size=pool_size))
my_model.add(BatchNormalization())


my_model.add(GRU(units = 8,dropout=dropout,recurrent_dropout=dropout))

my_model.add(Dense(1))



my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()


filepath="weights.{epoch:02d}-{val_loss:.2f}.hdf5"

callbacks = [
    #EarlyStopping(monitor='val_loss', patience=30, verbose=0),
    ModelCheckpoint(filepath, monitor='val_mean_absolute_error', verbose=1, 
                    save_best_only=True, mode='min')
]

my_training_batch_generator = MY_Generator(train_data_batch, NUMBER_OF_BATCHES, 'train')
my_validation_batch_generator = MY_Generator(val_data_batch, NUMBER_OF_VALIDATION_STEPS, 'val')


history = my_model.fit_generator(generator=my_training_batch_generator,
                                          #steps_per_epoch=NUMBER_OF_BATCHES,
                                          epochs=1000,
                                          validation_data=my_validation_batch_generator,
                                          #validation_steps=NUMBER_OF_VALIDATION_STEPS,
                                          callbacks=callbacks,
                                          shuffle=True,
                                          #verbose=1,
                                          #validation_data=my_validation_batch_generator,
                                          #validation_steps=(num_validation_samples // batch_size),
                                          use_multiprocessing=True,
                                          workers=8,
                                          #max_queue_size=32
                      )


import matplotlib.pyplot as plt


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#import math
#print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 30000, 3)          18        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 15000, 3)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15000, 3)          12        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 3000, 3)           48        
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1500, 3)           0         
_________________________________________________________________
batch_normalization_2

236/236 [==============================] - 50s 212ms/step - loss: 2.3783 - mean_absolute_error: 2.3508 - val_loss: 2.3493 - val_mean_absolute_error: 2.3222

Epoch 00020: val_mean_absolute_error improved from 2.34114 to 2.32222, saving model to weights.20-2.35.hdf5
Epoch 21/1000
236/236 [==============================] - 50s 211ms/step - loss: 2.3854 - mean_absolute_error: 2.3592 - val_loss: 2.3397 - val_mean_absolute_error: 2.3142

Epoch 00021: val_mean_absolute_error improved from 2.32222 to 2.31418, saving model to weights.21-2.34.hdf5
Epoch 22/1000
236/236 [==============================] - 50s 212ms/step - loss: 2.3790 - mean_absolute_error: 2.3540 - val_loss: 2.5623 - val_mean_absolute_error: 2.5381

Epoch 00022: val_mean_absolute_error did not improve from 2.31418
Epoch 23/1000
236/236 [==============================] - 50s 211ms/step - loss: 2.3781 - mean_absolute_error: 2.3545 - val_loss: 2.3588 - val_mean_absolute_error: 2.3355

Epoch 00023: val_mean_absolute_error did not imp

236/236 [==============================] - 50s 214ms/step - loss: 2.2796 - mean_absolute_error: 2.2616 - val_loss: 2.2884 - val_mean_absolute_error: 2.2707

Epoch 00053: val_mean_absolute_error did not improve from 2.24205
Epoch 54/1000

236/236 [==============================] - 51s 214ms/step - loss: 2.2857 - mean_absolute_error: 2.2678 - val_loss: 2.2683 - val_mean_absolute_error: 2.2503

Epoch 00054: val_mean_absolute_error did not improve from 2.24205
Epoch 55/1000
236/236 [==============================] - 52s 218ms/step - loss: 2.3015 - mean_absolute_error: 2.2836 - val_loss: 2.2881 - val_mean_absolute_error: 2.2703

Epoch 00055: val_mean_absolute_error did not improve from 2.24205
Epoch 56/1000
236/236 [==============================] - 50s 212ms/step - loss: 2.2818 - mean_absolute_error: 2.2643 - val_loss: 2.5526 - val_mean_absolute_error: 2.5352

Epoch 00056: val_mean_absolute_error did not improve from 2.24205
Epoch 57/1000
236/236 [==============================] - 51s 214m


Epoch 00084: val_mean_absolute_error did not improve from 2.18698
Epoch 85/1000
236/236 [==============================] - 51s 215ms/step - loss: 2.2679 - mean_absolute_error: 2.2501 - val_loss: 2.4238 - val_mean_absolute_error: 2.4065

Epoch 00085: val_mean_absolute_error did not improve from 2.18698
Epoch 86/1000
236/236 [==============================] - 51s 218ms/step - loss: 2.3073 - mean_absolute_error: 2.2894 - val_loss: 2.2399 - val_mean_absolute_error: 2.2218

Epoch 00086: val_mean_absolute_error did not improve from 2.18698
Epoch 87/1000
236/236 [==============================] - 50s 210ms/step - loss: 2.2539 - mean_absolute_error: 2.2359 - val_loss: 2.3389 - val_mean_absolute_error: 2.3207

Epoch 00087: val_mean_absolute_error did not improve from 2.18698
Epoch 88/1000
236/236 [==============================] - 50s 213ms/step - loss: 2.2869 - mean_absolute_error: 2.2689 - val_loss: 2.4535 - val_mean_absolute_error: 2.4356

Epoch 00088: val_mean_absolute_error did not improv

236/236 [==============================] - 51s 215ms/step - loss: 2.2568 - mean_absolute_error: 2.2388 - val_loss: 2.2786 - val_mean_absolute_error: 2.2609

Epoch 00119: val_mean_absolute_error did not improve from 2.18698
Epoch 120/1000
236/236 [==============================] - 50s 213ms/step - loss: 2.2463 - mean_absolute_error: 2.2286 - val_loss: 2.4330 - val_mean_absolute_error: 2.4151

Epoch 00120: val_mean_absolute_error did not improve from 2.18698
Epoch 121/1000
236/236 [==============================] - 50s 212ms/step - loss: 2.2457 - mean_absolute_error: 2.2277 - val_loss: 2.3244 - val_mean_absolute_error: 2.3066

Epoch 00121: val_mean_absolute_error did not improve from 2.18698
Epoch 122/1000
236/236 [==============================] - 51s 214ms/step - loss: 2.2743 - mean_absolute_error: 2.2562 - val_loss: 2.2563 - val_mean_absolute_error: 2.2381

Epoch 00122: val_mean_absolute_error did not improve from 2.18698
Epoch 123/1000
236/236 [==============================] - 51s 2

...

236/236 [==============================] - 51s 216ms/step - loss: 2.1583 - mean_absolute_error: 2.1389 - val_loss: 2.1876 - val_mean_absolute_error: 2.1684

Epoch 00525: val_mean_absolute_error did not improve from 2.15539
Epoch 526/1000
236/236 [==============================] - 52s 219ms/step - loss: 2.1615 - mean_absolute_error: 2.1421 - val_loss: 2.3516 - val_mean_absolute_error: 2.3323

Epoch 00526: val_mean_absolute_error did not improve from 2.15539
Epoch 527/1000
236/236 [==============================] - 51s 216ms/step - loss: 2.1428 - mean_absolute_error: 2.1231 - val_loss: 2.1944 - val_mean_absolute_error: 2.1748

Epoch 00527: val_mean_absolute_error did not improve from 2.15539
Epoch 528/1000
236/236 [==============================] - 50s 212ms/step - loss: 2.1610 - mean_absolute_error: 2.1416 - val_loss: 2.2319 - val_mean_absolute_error: 2.2121

Epoch 00528: val_mean_absolute_error did not improve from 2.15539
Epoch 529/1000
236/236 [==============================] - 51s 2

236/236 [==============================] - 51s 217ms/step - loss: 2.1561 - mean_absolute_error: 2.1344 - val_loss: 2.4725 - val_mean_absolute_error: 2.4510

Epoch 00559: val_mean_absolute_error did not improve from 2.15539
Epoch 560/1000
236/236 [==============================] - 50s 213ms/step - loss: 2.1575 - mean_absolute_error: 2.1361 - val_loss: 2.2829 - val_mean_absolute_error: 2.2615

Epoch 00560: val_mean_absolute_error did not improve from 2.15539
Epoch 561/1000
236/236 [==============================] - 51s 214ms/step - loss: 2.1551 - mean_absolute_error: 2.1337 - val_loss: 2.7161 - val_mean_absolute_error: 2.6943

Epoch 00561: val_mean_absolute_error did not improve from 2.15539
Epoch 562/1000
236/236 [==============================] - 51s 217ms/step - loss: 2.1559 - mean_absolute_error: 2.1343 - val_loss: 2.2066 - val_mean_absolute_error: 2.1850

Epoch 00562: val_mean_absolute_error did not improve from 2.15539
Epoch 563/1000
236/236 [==============================] - 51s 2

236/236 [==============================] - 51s 216ms/step - loss: 2.1465 - mean_absolute_error: 2.1254 - val_loss: 4.0097 - val_mean_absolute_error: 3.9882

Epoch 00593: val_mean_absolute_error did not improve from 2.15539
Epoch 594/1000
236/236 [==============================] - 51s 215ms/step - loss: 2.1596 - mean_absolute_error: 2.1382 - val_loss: 2.2057 - val_mean_absolute_error: 2.1846

Epoch 00594: val_mean_absolute_error did not improve from 2.15539
Epoch 595/1000

Epoch 00593: val_mean_absolute_error did not improve from 2.15539
Epoch 594/1000
236/236 [==============================] - 51s 215ms/step - loss: 2.1512 - mean_absolute_error: 2.1302 - val_loss: 2.2799 - val_mean_absolute_error: 2.2586

Epoch 00595: val_mean_absolute_error did not improve from 2.15539
Epoch 596/1000
236/236 [==============================] - 50s 213ms/step - loss: 2.1497 - mean_absolute_error: 2.1285 - val_loss: 2.5486 - val_mean_absolute_error: 2.5277

Epoch 00596: val_mean_absolute_error did not im

236/236 [==============================] - 50s 212ms/step - loss: 2.1621 - mean_absolute_error: 2.1412 - val_loss: 2.3390 - val_mean_absolute_error: 2.3180

Epoch 00627: val_mean_absolute_error did not improve from 2.15539
Epoch 628/1000
236/236 [==============================] - 51s 218ms/step - loss: 2.1449 - mean_absolute_error: 2.1236 - val_loss: 2.3417 - val_mean_absolute_error: 2.3203

Epoch 00628: val_mean_absolute_error did not improve from 2.15539
Epoch 629/1000
236/236 [==============================] - 51s 218ms/step - loss: 2.1526 - mean_absolute_error: 2.1314 - val_loss: 3.2388 - val_mean_absolute_error: 3.2178

Epoch 00629: val_mean_absolute_error did not improve from 2.15539
Epoch 630/1000
236/236 [==============================] - 51s 214ms/step - loss: 2.1555 - mean_absolute_error: 2.1342 - val_loss: 2.3094 - val_mean_absolute_error: 2.2881

Epoch 00630: val_mean_absolute_error did not improve from 2.15539
Epoch 631/1000
236/236 [==============================] - 51s 2


Epoch 00661: val_mean_absolute_error did not improve from 2.15539
Epoch 662/1000
236/236 [==============================] - 52s 218ms/step - loss: 2.1537 - mean_absolute_error: 2.1329 - val_loss: 2.2340 - val_mean_absolute_error: 2.2135

Epoch 00662: val_mean_absolute_error did not improve from 2.15539
Epoch 663/1000
236/236 [==============================] - 50s 213ms/step - loss: 2.1474 - mean_absolute_error: 2.1271 - val_loss: 3.0490 - val_mean_absolute_error: 3.0287

Epoch 00663: val_mean_absolute_error did not improve from 2.15539
Epoch 664/1000
236/236 [==============================] - 50s 211ms/step - loss: 2.1481 - mean_absolute_error: 2.1275 - val_loss: 2.9190 - val_mean_absolute_error: 2.8982

Epoch 00664: val_mean_absolute_error did not improve from 2.15539
Epoch 665/1000
236/236 [==============================] - 50s 213ms/step - loss: 2.1505 - mean_absolute_error: 2.1296 - val_loss: 2.4008 - val_mean_absolute_error: 2.3799

Epoch 00665: val_mean_absolute_error did not im

In [ ]:
TEST_SPLITS='test'
test_splits = [f for f in listdir(TEST_SPLITS) if isfile(join(TEST_SPLITS, f))]

In [ ]:
test_splits

In [ ]:
test_split_chunks = np.array_split(test_splits,mp.cpu_count())

import build_segment
importlib.reload(build_segment)

from build_segment import build_segment_f

if __name__ ==  '__main__':
    pool = mp.Pool(mp.cpu_count())
    res = [pool.apply_async(build_segment_f,args=[chunk,TIMESTEPS, True]) \
           for chunk in test_split_chunks]
    pool.close()
    pool.join()

In [ ]:
ids = []
preds = []
i=0
for r in res:
    for df in r.get():
        if i % 100 == 0:
            print(i)
        #training[i] = df.loc[:,df.columns != 'time_to_failure']
        ids.append(df['seg_id'].unique()[0].split(".")[0])
        test_df = df.drop('seg_id', axis=1)
        preds.append(my_model.predict(test_df.values.reshape(1,-1,NUMBER_OF_FEATURES))[0][0])
        i+=1

In [ ]:
submission = pd.DataFrame(ids)
submission.columns = ['seg_id']
submission['time_to_failure'] = preds

submission.to_csv('submission.csv', index=False)

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
submission["time_to_failure"].describe()

In [ ]:
submission.head()

In [ ]:
preds

In [ ]:
res[0].get()[0]

In [ ]:
path = 'test/%s' % (np.random.choice(test_splits))
#

df = pd.read_csv(path, float_precision='round_trip', header=[0])


df.describe()